# Coarse-graining at amino acid level of all-atom structures

In [1]:
### Import all packages needed
import mdtraj as md
import numpy as np
import os

workdir = '/Users/isabelvinterbladh/Documents/YRI-Talk/YRI-Protein-Simulation' # Change to your working directory

### Check what your working directory is (your current directory)
!pwd
### Copy the output from this cell to the workdir definition above

/Users/isabelvinterbladh/Documents/YRI-Talk/YRI-Protein-Simulation


In [2]:
### Go to the working directory
%cd $workdir

/Users/isabelvinterbladh/Documents/YRI-Talk/YRI-Protein-Simulation


# Steps:

1. Perform the CG at amino acid level
2. Rename terminals 
3. If applicable - rename cysteine involved in disulfide bonds to avoid that they are titratable during simulation

###  1. CG at amino acid level

In [3]:
### Load complete pdb structure ###

def CGfunc(traj, file_xyz):
    amino_acids = ['ALA','ARG','ASN','ASP','CYS','GLN','GLU','GLY','HIS','ILE','LEU','LYS','MET','PHE','PRO','SER','THR','TRP','TYR','VAL']
    num_amino_acids = sum(1 for res in traj.topology.residues if res.name in amino_acids)
    num_chains = sum(1 for chain in traj.topology.chains if any(res.name in amino_acids for res in chain.residues))
    print('')
    print('The pdb structure contains '+str(traj.topology.n_atoms)+' atoms, '+str(num_amino_acids)+' amino acids, and '+str(num_chains)+' chains!')
    print('Coarse graining at amino acid level ...')
        
    file = open(file_xyz, 'w')
    file.write(str(num_amino_acids)+'\n')
    file.write('\n')
    for res in traj.topology.residues:    
        cm = [0,0,0] # residue mass center
        mw = 0       # residue weight
        if res.name not in amino_acids:
            continue
        else:
            for a in res.atoms:
                cm = cm + a.element.mass * traj.xyz[0][a.index]
                mw = mw + a.element.mass
            cm = cm/mw*10
            file.write('{0:4} {1:8.3f} {2:8.3f} {3:8.3f}\n'.format(res.name, cm[0],cm[1],cm[2]))
    file.close()
    
    print('Done !!!')
    print('The xyz-file has been created!')

### 2. Renaming the chain terminals/ends

In [4]:
def fix_terminals(traj, file_xyz):
    cterm = []
    nterm = []
    amino_acids = ['ALA','ARG','ASN','ASP','CYS','GLN','GLU','GLY','HIS','ILE','LEU','LYS','MET','PHE','PRO','SER','THR','TRP','TYR','VAL']

    # Find the indices and amino acid names for the terminals
    for chain in traj.topology.chains:
        res_list = list(chain.residues)
        if res_list[0].name not in amino_acids:
            continue
        else:
            first = res_list[0].index, res_list[0].name 
            last = res_list[-1].index, res_list[-1].name 
            cterm.append(first)
            nterm.append(last)  
    file = open(file_xyz, 'r+') 
    lines = file.readlines() # Read the entire file into a list of lines
    # name the aas added to cterm to CTR    
    for c in cterm:
        line = lines[c[0]+2]
        mod_line = line.replace( c[1] , 'CTR', 1)
        lines[c[0]+2] = mod_line
        with open(file_xyz, 'w') as file:
            for line in lines:
                file.write(line)
    # name the aas added to nterm to NTR               
    for n in nterm:
        line = lines[n[0]+2]
        mod_line = line.replace(n[1] , 'NTR', 1)
        lines[n[0]+2] = mod_line
        with open(file_xyz, 'w') as file:
            for line in lines:
                file.write(line) 
    file.close()   


### 3. Disulfate bond
Find the disulfate bonds and thereafter rename them in the xyz file.

In [5]:
def SS_search(traj, file_xyz):
    print("Searching cysteine involved in disulfide bonds ...")
    print('')
    ### Search for ss bonds ###
    SS_bonds = np.empty(0, dtype=int)
    for b in traj.topology.bonds:
        i = b[0].residue.index
        j = b[1].residue.index
        if (b[0].residue.name == 'CYS'):
            if (b[1].residue.name == 'CYS'):
                if i < traj.topology.n_residues:
                    if j > i + 1:
                        print('SS-bond between residues', i, j)
                        SS_bonds = np.append(SS_bonds, [i+2,j+2], axis=0)
    print('')
    print('There are '+str(np.size(SS_bonds))+' amino acids involved in disulfide bonds.' )
    print('Indexes of cysteines involved in disulfide bonds: '+str(SS_bonds))

    ### Renaming CYS residues as CYX ###
    print('Renaming residues involved in disulfide bonds ...')

    fin = open(file_xyz, 'rt')
    fout = open('file_out.xyz', 'wt')

    for h,line in enumerate(fin):

        if h in SS_bonds:
            line = line.replace("CYS", "CYX")
        fout.write(line)  

    fin.close()      
    fout.close()
    os.rename('file_out.xyz', file_xyz)
    print('Done !!!')

## Script creating final CG xyz-file

In [8]:
def create_xyzfile(pdb, file_xyz, Duello, SS_bonds = False):
    traj = md.load_pdb(pdb)
    CGfunc(traj, file_xyz)
    if Duello is False:
        fix_terminals(traj, file_xyz)
    if SS_bonds is True:
        SS_search(traj, file_xyz)

%cd $workdir/structures

PDB = '2hiu.pdb'  ## Change to name of pdb file
output_file = '2hiu.xyz' ## Change to desired output file name for the xyz-file

### Run the script!
# if SS-bonds present, add True to function input
# if you want to create a xyz file for Duello, change Duello to True

create_xyzfile(PDB, output_file, Duello = False, SS_bonds = True) 

/Users/isabelvinterbladh/Documents/YRI-Talk/YRI-Protein-Simulation/structures

The pdb structure contains 786 atoms, 51 amino acids, and 2 chains!
Coarse graining at amino acid level ...
Done !!!
The xyz-file has been created!
Searching cysteine involved in disulfide bonds ...

SS-bond between residues 5 10
SS-bond between residues 6 27
SS-bond between residues 19 39

There are 6 amino acids involved in disulfide bonds.
Indexes of cysteines involved in disulfide bonds: [ 7 12  8 29 21 41]
Renaming residues involved in disulfide bonds ...
Done !!!
